In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
This is a sample RNN code.
Using this original code, you could already get a 0.63 accuracy.
Please make some changes instead of submitting the original code.
If you have some questions about the code or data,
please contact M.M. Kuang(kuangmeng@hku.hk) or Y.Q. Deng(yqdeng@cs.hku.hk)
"""
import time
import sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

map_fn = tf.map_fn

In [2]:
#new
def str_2_list(data_list):
    ret_list = []
    for i in range(len(data_list)):
        tmp_list = data_list[i].strip().split(" ")
        tmp_ret_list = [int(tmp_list[7][0]),int(tmp_list[6]),int(tmp_list[5]),int(tmp_list[4]),int(tmp_list[3]),int(tmp_list[2]),int(tmp_list[1]),int(tmp_list[0][1])] #changed
        ret_list.append(tmp_ret_list)
    return ret_list

# Prepare Data(Training and Testing)
filename = "data.txt"
a_list = []
b_list = []
c_list = []


In [3]:
with open(filename, "r") as file:
    filein = file.read().splitlines()
    for item in filein:
        tmp_list = item.strip().split(",")
        a_list.append(tmp_list[0])
        b_list.append(tmp_list[1])
        c_list.append(tmp_list[2])
a_list = str_2_list(a_list)
b_list = str_2_list(b_list)
c_list = str_2_list(c_list)

In [4]:
# Define the dataflow graph
time_steps = 8        # time steps which is the same as the length of the bit-string
input_dim = 2         # number of units in the input layer
hidden_dim = 24      # number of units in the hidden layer
output_dim = 1        # number of units in the output layer
binary_dim = 8
largest_number = pow(2, binary_dim)
TINY          = 1e-6    # to avoid NaNs in logs

In [5]:
tf.reset_default_graph()
# input inputs and target ouput Y
inputs = tf.placeholder(tf.float32, [time_steps, None, input_dim], name='x')
outputs = tf.placeholder(tf.float32, [time_steps, None, output_dim], name='y')

# define the RNN cell: can be simple cell, LSTM or GRU
cell = tf.contrib.rnn.BasicLSTMCell(hidden_dim, state_is_tuple=True)
# cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim, activation=tf.nn.tanh)

batch_size    = tf.shape(inputs)[1]
initial_state = cell.zero_state(batch_size, tf.float32)

# Given inputs (time, batch, input_size) outputs a tuple
#  - outputs: (time, batch, output_size)  [do not mistake with output_dim]
#  - states:  (time, batch, hidden_size)
rnn_outputs, rnn_states = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, time_major=True)

# project output from rnn output size to output_dim. Sometimes it is worth adding
# an extra layer here.
final_projection = lambda x: layers.linear(x, num_outputs=output_dim, activation_fn=tf.nn.sigmoid)

# apply projection to every timestep.
predicted_outputs = map_fn(final_projection, rnn_outputs)

# # minimize error, using ADAM as weight update rule
error = -(outputs * tf.log(predicted_outputs + TINY) + (1.0 - outputs) * tf.log(1.0 - predicted_outputs + TINY))
error = tf.reduce_sum(error)

learning_rate = 0.06
global_step = tf.Variable(0, trainable=False)
decayed_lr = tf.train.exponential_decay(learning_rate, global_step, 100000, 0.99, staircase=True)
# train_fn = tf.train.AdamOptimizer(decayed_lr).minimize(error, global_step=global_step)
train_fn = tf.train.AdamOptimizer(learning_rate).minimize(error)
accuracy = tf.reduce_mean(tf.cast(tf.abs(outputs - predicted_outputs) < 0.5, tf.float32))

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Number of data for training and testing
#Please remember the total number is 5000
num4train = 3000
num4valid = 1000
num4test = 1000

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# train
batch_size = 500
early_stopping_tolerance = 2
early_stopping_counter = 0
min_valid_error = sys.maxsize


valid_a = np.transpose([a_list[num4train: num4train + num4valid]])
valid_b = np.transpose([b_list[num4train: num4train + num4valid]])
valid_y = np.transpose([c_list[num4train: num4train + num4valid]])
valid_x = np.concatenate((valid_a,valid_b), axis=2)

num_of_epochs = 0
start_time = time.time()
while True:
    num_of_epochs = num_of_epochs + 1
    train_error = 0
    ITERATIONS_PER_EPOCH = int(num4train/batch_size)
    for j in range(ITERATIONS_PER_EPOCH):
        a = np.transpose([a_list[j*batch_size : (j+1)*batch_size]])
        b = np.transpose([b_list[j*batch_size : (j+1)*batch_size]])
        y = np.transpose([c_list[j*batch_size : (j+1)*batch_size]])
        x = np.concatenate((a,b), axis=2)

        train_error += sess.run([error, train_fn], {
            inputs: x,
            outputs: y,
        })[0]
    train_error /= ITERATIONS_PER_EPOCH
    valid_accuracy = sess.run(accuracy, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    valid_error = sess.run(error, {
        inputs:  valid_x,
        outputs: valid_y,
    })
    print("Epoch ", num_of_epochs, ", train err: ", train_error, "valid err: ", valid_error, ", valid acc: ", valid_accuracy * 100.0)
    #early stopping
    if valid_error < 1:
            #early stop
            break
            
#     if valid_error < min_valid_error:
#         min_valid_error = valid_error
#         early_stopping_counter = 0
#     else:
#         early_stopping_counter = early_stopping_counter + 1
#         print("early_stopping_counter: " + str(early_stopping_counter))
#         if early_stopping_counter >= early_stopping_tolerance:
#             #early stop
#             break
end_time = time.time()

remain_result = []
print("Training Finished")
print("Training time: ", end_time - start_time)

Epoch  1 , train err:  2864.5542805989585 valid err:  5597.4355 , valid acc:  53.737497329711914
Epoch  2 , train err:  2772.179484049479 valid err:  5510.4746 , valid acc:  54.362499713897705
Epoch  3 , train err:  2661.641805013021 valid err:  5345.2227 , valid acc:  66.8749988079071
Epoch  4 , train err:  2618.181844075521 valid err:  5220.3086 , valid acc:  74.9750018119812
Epoch  5 , train err:  2574.8702799479165 valid err:  5100.9033 , valid acc:  71.79999947547913
Epoch  6 , train err:  2490.4158935546875 valid err:  4917.605 , valid acc:  70.98749876022339
Epoch  7 , train err:  2435.464070638021 valid err:  4684.457 , valid acc:  72.54999876022339
Epoch  8 , train err:  2276.3076985677085 valid err:  4109.6836 , valid acc:  81.17499947547913
Epoch  9 , train err:  2082.7217000325522 valid err:  3360.901 , valid acc:  81.88750147819519
Epoch  10 , train err:  1733.5636800130208 valid err:  2560.5366 , valid acc:  87.40000128746033
Epoch  11 , train err:  1519.5357666015625 val

In [7]:
remain_result = []
average_error = 0
#Test
for i in range(num4train + num4valid + 1, num4train + num4valid + num4test):
    a = np.transpose([[a_list[i]]])
    b = np.transpose([[b_list[i]]])
    y = np.transpose([[c_list[i]]])
    x = np.concatenate((a,b), axis=2)

    # get predicted value
    [_probs, _error] = sess.run([predicted_outputs, error], {inputs: x, outputs: y})
    probs = np.array(_probs).reshape([8])
    prediction = np.array([1 if p >= 0.5 else 0 for p in probs]).reshape([8])
    # Save the result
    disired_output = y.reshape([binary_dim])
    remain_result.append([prediction, disired_output])

    # calculate error
    y = y.reshape(binary_dim)
    probs = probs.reshape(binary_dim)
    error_value = np.sum(np.absolute(y - probs))
    
    #print the prediction, the right y and the error.
    print("---------------")
    print(prediction)
    print(disired_output)
    print(error_value)
    print("---------------")
    print()

average_error = average_error/num4test
print(average_error)

---------------
[0 0 0 1 0 1 0 1]
[0 0 0 1 0 1 0 1]
0.0004922412808809895
---------------

---------------
[0 0 1 1 1 0 0 1]
[0 0 1 1 1 0 0 1]
0.00028983328297726985
---------------

---------------
[0 1 1 0 1 0 0 1]
[0 1 1 0 1 0 0 1]
0.0008846874102346192
---------------

---------------
[0 0 0 0 1 0 0 1]
[0 0 0 0 1 0 0 1]
0.00048456828108101035
---------------

---------------
[0 0 1 0 0 0 0 1]
[0 0 1 0 0 0 0 1]
0.0009982581141230185
---------------

---------------
[0 1 1 1 1 1 1 0]
[0 1 1 1 1 1 1 0]
0.0007960457296576351
---------------

---------------
[0 0 0 1 1 1 1 0]
[0 0 0 1 1 1 1 0]
0.000739114846510347
---------------

---------------
[0 1 0 0 1 1 1 0]
[0 1 0 0 1 1 1 0]
0.0006610631899093278
---------------

---------------
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
0.0007228771683003288
---------------

---------------
[0 0 0 1 0 1 1 1]
[0 0 0 1 0 1 1 1]
0.0005533141047635581
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.0004414946524775587
---------------

---------------
[0 1 0 1 1 1 1 0]
[0 1 0 1 1 1 1 0]
0.0007449779404851142
---------------

---------------
[0 0 1 0 1 1 1 0]
[0 0 1 0 1 1 1 0]
0.00046682304309797473
---------------

---------------
[0 1 1 1 0 1 1 0]
[0 1 1 1 0 1 1 0]
0.000789383273513522
---------------

---------------
[0 0 0 1 0 1 1 0]
[0 0 0 1 0 1 1 0]
0.0006436514413508121
---------------

---------------
[0 0 1 1 1 0 1 0]
[0 0 1 1 1 0 1 0]
0.00033125519985333085
---------------

---------------
[0 1 1 0 1 0 1 0]
[0 1 1 0 1 0 1 0]
0.0007699969196437451
---------------

---------------
[0 0 0 0 1 0 1 0]
[0 0 0 0 1 0 1 0]
0.0007002868314884836
---------------

---------------
[0 1 0 1 0 0 1 0]
[0 1 0 1 0 0 1 0]
0.0011491357035993133
---------------

---------------
[0 0 1 1 1 1 1 1]
[0 0 1 1 1 1 1 1]
0.0006919575534993783
---------------

---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.0005495384648384061
---------------

---------------
[0 0 0 1 1 0 1 1]
[0 0 0 1 1 0 1 1]
0.0007426339607263799
---------------

---------------
[0 0 1 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
0.0004024938061775174
---------------

---------------
[0 1 0 1 1 1 1 1]
[0 1 0 1 1 1 1 1]
0.0008147542139340658
---------------

---------------
[0 0 1 0 1 1 1 1]
[0 0 1 0 1 1 1 1]
0.0006541456350532826
---------------

---------------
[0 1 1 1 0 1 1 1]
[0 1 1 1 0 1 1 1]
0.0006768538078176789
---------------

---------------
[0 1 0 0 0 1 1 1]
[0 1 0 0 0 1 1 1]
0.00072123116694911
---------------

---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00040167874976759776
---------------

---------------
[0 1 1 0 1 0 1 1]
[0 1 1 0 1 0 1 1]
0.0006996115321271645
---------------

---------------
[0 0 0 0 1 0 1 1]
[0 0 0 0 1 0 1 1]
0.0004847517211601371
---------------

---------------
[0 1 0 1 0 0 1 1]
[0 1 0 1 0 0 1 1]
0.0013637204283440951
---------------

---------------
[0 1 1 1 1 1 0 1]
[0 1 1 1 1 1 0 1]
0.0011621718876995146
---------------

---------------
[0 0 0 1 1 1 0 1]
[0 0 0 1 1 1 0 1]
0.0004961429367540404
---------------



---------------
[0 1 1 0 1 0 1 1]
[0 1 1 0 1 0 1 1]
0.0006996115321271645
---------------

---------------
[0 0 0 0 1 0 1 1]
[0 0 0 0 1 0 1 1]
0.0004847517211601371
---------------

---------------
[0 1 0 1 0 0 1 1]
[0 1 0 1 0 0 1 1]
0.0013637204283440951
---------------

---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.000463371373029986
---------------

---------------
[0 1 1 1 1 1 0 1]
[0 1 1 1 1 1 0 1]
0.0011621718876995146
---------------

---------------
[0 1 0 0 1 1 0 1]
[0 1 0 0 1 1 0 1]
0.0005160900727787521
---------------

---------------
[0 0 1 1 0 1 0 1]
[0 0 1 1 0 1 0 1]
0.00045680297807848547
---------------

---------------
[0 1 1 0 0 1 0 1]
[0 1 1 0 0 1 0 1]
0.0017910508013301296
---------------

---------------
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 1 0 1]
0.0005222262952884194
---------------

---------------
[0 1 0 1 1 0 0 1]
[0 1 0 1 1 0 0 1]
0.0015731725106888916
---------------

---------------
[0 1 1 1 0 0 0 1]
[0 1 1 1 0 0 0 1]
0.0004096874422430119
---------------


---------------
[0 0 1 1 1 0 1 1]
[0 0 1 1 1 0 1 1]
0.00040167874976759776
---------------

---------------
[0 1 1 0 1 0 1 1]
[0 1 1 0 1 0 1 1]
0.0006996115321271645
---------------

---------------
[0 1 0 1 0 0 1 1]
[0 1 0 1 0 0 1 1]
0.0013637204283440951
---------------

---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.000463371373029986
---------------

---------------
[0 1 1 1 1 1 0 1]
[0 1 1 1 1 1 0 1]
0.0011621718876995146
---------------

---------------
[0 0 0 1 1 1 0 1]
[0 0 0 1 1 1 0 1]
0.0004961429367540404
---------------

---------------
[0 1 0 0 1 1 0 1]
[0 1 0 0 1 1 0 1]
0.0005160900727787521
---------------

---------------
[0 1 1 0 0 1 0 1]
[0 1 1 0 0 1 0 1]
0.0017910508013301296
---------------

---------------
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 1 0 1]
0.0005222262952884194
---------------

---------------
[0 1 0 1 1 0 0 1]
[0 1 0 1 1 0 0 1]
0.0015731725106888916
---------------

---------------
[0 0 1 0 1 0 0 1]
[0 0 1 0 1 0 0 1]
0.0008271420338132884
---------------


---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.0008078862198317438
---------------

---------------
[0 0 0 1 1 1 0 1]
[0 0 0 1 1 1 0 1]
0.00038349015812855214
---------------

---------------
[0 0 1 1 0 1 0 1]
[0 0 1 1 0 1 0 1]
0.0004090000529686222
---------------

---------------
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 1 0 1]
0.00045368095379672013
---------------

---------------
[0 0 1 0 1 0 0 1]
[0 0 1 0 1 0 0 1]
0.000813194546935847
---------------

---------------
[0 0 1 1 0 1 1 1]
[0 0 1 1 0 1 1 1]
0.0006099300608184421
---------------

---------------
[0 0 0 0 0 1 1 1]
[0 0 0 0 0 1 1 1]
0.00039807807115721516
---------------

---------------
[0 0 1 0 1 0 1 1]
[0 0 1 0 1 0 1 1]
0.0003882556593453046
---------------

---------------
[0 0 0 1 0 0 1 1]
[0 0 0 1 0 0 1 1]
0.00041143990711134393
---------------

---------------
[0 0 1 1 1 1 0 1]
[0 0 1 1 1 1 0 1]
0.0005991996004013345
---------------

---------------
[0 0 0 0 1 1 1 1]
[0 0 0 0 1 1 1 1]
0.0005021984052291373
-------------

---------------
[0 0 1 0 0 0 1 1]
[0 0 1 0 0 0 1 1]
0.000463371373029986
---------------

---------------
[0 1 1 1 1 1 0 1]
[0 1 1 1 1 1 0 1]
0.0011621718876995146
---------------

---------------
[0 1 0 0 1 1 0 1]
[0 1 0 0 1 1 0 1]
0.0005160900727787521
---------------

---------------
[0 0 1 1 0 1 0 1]
[0 0 1 1 0 1 0 1]
0.00045680297807848547
---------------

---------------
[0 1 1 0 0 1 0 1]
[0 1 1 0 0 1 0 1]
0.0017910508013301296
---------------

---------------
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 1 0 1]
0.0005222262952884194
---------------

---------------
[0 1 0 1 1 0 0 1]
[0 1 0 1 1 0 0 1]
0.0015731725106888916
---------------

---------------
[0 1 1 1 0 0 0 1]
[0 1 1 1 0 0 0 1]
0.0004096874422430119
---------------

---------------
[0 0 0 1 0 0 0 1]
[0 0 0 1 0 0 0 1]
0.0004420007570047346
---------------

---------------
[0 1 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1]
0.001956667227204889
---------------

---------------
[0 0 1 1 1 1 1 0]
[0 0 1 1 1 1 1 0]
0.0009149261895800009
---------------



In [8]:
# Get the total accuracy (Please don't change this part)
accuracy = 0
for i in range(len(remain_result)):
    len_ = len(remain_result[i][0])
    tmp_num = 0
    for j in range(len_):
        if remain_result[i][0][j] == remain_result[i][1][j]:
            tmp_num += 1
    accuracy += tmp_num / len_

accuracy /= len(remain_result)

print("Accuracy: %.4f"%(accuracy))

Accuracy: 1.0000


In [149]:
sess.close()